<a href="https://colab.research.google.com/github/A00785001/TC5035/blob/main/000-ROS_Bag_Data_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROS Bag Data Extractor
Extract and analyze data from ROS bags

## <font color='#2E86AB'>▼ 1. Initialization and Setup</font>

In [ ]:
# Install required packages
!pip install --quiet rosbags opencv-python pillow numpy matplotlib tqdm seaborn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 18.3 MB/s eta 0:00:00


In [ ]:
# Import libraries
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict
import io

print("Libraries loaded successfully!")

Libraries loaded successfully!


In [ ]:
from rosbags.rosbag1 import Reader
from rosbags.typesys import Stores, get_typestore
import os
from pathlib import Path
print("Libraries loaded successfully!")

Libraries loaded successfully!


In [ ]:
# Initialize typestore for ROS1 message deserialization
typestore = get_typestore(Stores.ROS1_NOETIC)
print("Typestore initialized for ROS1")

Typestore initialized for ROS1


## <font color='#2E86AB'>▼ 2. Storage Mounting</font>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## <font color='#2E86AB'>▼ 3. ROS Bag Load</font>

In [ ]:
# Specify the path to the sessions folder
data_path = "/content/drive/MyDrive/DATA/Artificial_Intelligence/MNA-V/Subjects/TC5035-Proyecto_Integrador/TC5035.data/jetbot/"

In [ ]:
# Specify the session
session = '20251016_133216'
print(f"Using session: {session}")

Using session: 20251016_133216


In [ ]:
working_folder = data_path + 'session_' + session
bag_name = 'session_data.bag'

# Change to the specified subfolder
os.chdir(working_folder)
print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /content/drive/MyDrive/DATA/Artificial_Intelligence/MNA-V/Subjects/TC5035-Proyecto_Integrador/TC5035.data/jetbot/session_20251016_133216


In [ ]:
bag_file = working_folder + '/' + bag_name
print(f"Bag file name: {bag_name}")

# Set bag_path for the rest of the notebook
bag_path = bag_file

Bag file name: session_data.bag


## <font color='#2E86AB'>▼ 4. ROS Bag Basic EDA</font>

### 1. Basic Bag Information

In [ ]:
# Open the bag and get basic information
print("=" * 60)
print("BASIC BAG INFORMATION")
print("=" * 60)

with Reader(bag_path) as reader:
    # Get basic statistics
    duration = reader.duration * 1e-9  # Convert to seconds
    duration_sec = duration
    total_messages = reader.message_count

    # Get file size
    file_size_bytes = os.path.getsize(bag_path)
    file_size_mb = file_size_bytes / (1024 * 1024)

    # Calculate average message rate
    avg_message_rate = total_messages / duration if duration > 0 else 0

    print(f"\n📁 File: {bag_name}")
    print(f"💾 Size: {file_size_mb:.2f} MB")
    print(f"⏱️  Duration: {duration:.2f} seconds ({duration/60:.2f} minutes)")
    print(f"📬 Total Messages: {total_messages:,}")
    print(f"📊 Average Rate: {avg_message_rate:.2f} msg/sec")

BASIC BAG INFORMATION

📁 File: session_data.bag
💾 Size: 34.35 MB
⏱️  Duration: 463.17 seconds (7.72 minutes)
📬 Total Messages: 98,205
📊 Average Rate: 212.03 msg/sec


### 2. Topic Analysis

In [ ]:
# Analyze topics and connections
print("=" * 60)
print("TOPIC ANALYSIS (ROS1 BAG)")
print("=" * 60)

import struct

def parse_tf_binary(rawdata):
    """Parse TF message from raw bytes"""
    transforms = set()
    for start_offset in [0, 4, 8, 12, 16]:
        try:
            offset = start_offset
            array_len = struct.unpack('<I', rawdata[offset:offset+4])[0]
            if array_len > 100 or array_len == 0:
                continue
            offset += 4

            for _ in range(min(array_len, 5)):
                offset += 12  # Skip seq + stamp
                str_len = struct.unpack('<I', rawdata[offset:offset+4])[0]
                if str_len > 100 or str_len == 0:
                    break
                offset += 4
                parent = rawdata[offset:offset+str_len].decode('utf-8', errors='ignore').strip('\\x00').strip('/')
                offset += str_len

                str_len = struct.unpack('<I', rawdata[offset:offset+4])[0]
                if str_len > 100 or str_len == 0:
                    break
                offset += 4
                child = rawdata[offset:offset+str_len].decode('utf-8', errors='ignore').strip('\\x00').strip('/')
                offset += str_len + 56  # Skip transform data

                if parent and child and len(parent) < 50 and len(child) < 50:
                    transforms.add(f"{parent}→{child}")
            if transforms:
                return transforms
        except:
            continue
    return transforms

# Get basic topic information with connection indices
with Reader(bag_path) as reader:
    topics_data = []
    connection_map = {}  # Store connection index mapping

    for conn_idx, connection in enumerate(reader.connections):
        topics_data.append({
            'Topic': connection.topic,
            'Message Type': connection.msgtype,
            'Count': connection.msgcount,
            'Connection_Index': conn_idx,
            'Description': ""
        })
        # Map for later use
        connection_map[conn_idx] = {'topic': connection.topic, 'msgtype': connection.msgtype, 'msgcount': connection.msgcount}

    df_topics = pd.DataFrame(topics_data)

# Analyze TF connections
print("\\n🔍 Analyzing TF connections...")
with Reader(bag_path) as reader:
    tf_connection_data = {}

    for conn_idx, conn in enumerate(reader.connections):
        if conn.topic in ['/tf', '/tf_static']:
            tf_connection_data[conn_idx] = {'transforms': set(), 'samples': 0}

    for connection, timestamp, rawdata in reader.messages():
        if connection.topic in ['/tf', '/tf_static']:
            conn_idx = None
            for idx, conn in enumerate(reader.connections):
                if (conn.topic == connection.topic and
                    conn.msgtype == connection.msgtype and
                    conn.msgcount == connection.msgcount):
                    conn_idx = idx
                    break

            if conn_idx is not None and conn_idx in tf_connection_data:
                if tf_connection_data[conn_idx]['samples'] < 10:
                    try:
                        msg = typestore.deserialize_ros1(rawdata, connection.msgtype)
                        for tf in msg.transforms:
                            parent = tf.header.frame_id.strip('/')
                            child = tf.child_frame_id.strip('/')
                            tf_connection_data[conn_idx]['transforms'].add(f"{parent}→{child}")
                    except:
                        binary_transforms = parse_tf_binary(rawdata)
                        if binary_transforms:
                            tf_connection_data[conn_idx]['transforms'].update(binary_transforms)
                    tf_connection_data[conn_idx]['samples'] += 1

# Classify TF connections
for idx, row in df_topics.iterrows():
    if row['Topic'] in ['/tf', '/tf_static']:
        conn_idx = row['Connection_Index']

        if conn_idx in tf_connection_data:
            transforms = tf_connection_data[conn_idx]['transforms']

            if transforms:
                transforms_str = ', '.join(sorted(transforms)[:5])
                if len(transforms) > 5:
                    transforms_str += f" (+{len(transforms)-5} more)"

                transforms_lower = ' '.join(transforms).lower()

                if 'map' in transforms_lower and 'odom' in transforms_lower:
                    classification = "SLAM"
                elif 'odom' in transforms_lower and 'base' in transforms_lower:
                    classification = "Odometry"
                elif 'imu' in transforms_lower or 'laser' in transforms_lower or 'camera' in transforms_lower:
                    classification = "Static"
                else:
                    classification = "Other"

                df_topics.at[idx, 'Description'] = f"{classification}: {transforms_str}"
            else:
                freq = row['Count'] / duration_sec
                if freq > 40:
                    classification = "SLAM"
                elif freq > 30:
                    classification = "Odometry"
                elif freq > 15:
                    classification = "Static"
                else:
                    classification = "Other"
                df_topics.at[idx, 'Description'] = f"{classification} (by frequency)"

# Display
df_topics_display = df_topics.drop('Connection_Index', axis=1)
df_topics_display = df_topics_display.sort_values('Count', ascending=False)
df_topics_display['Frequency (Hz)'] = df_topics_display['Count'] / duration_sec

print(f"\\n📊 Total Connections: {len(df_topics_display)}")
print(f"📊 Unique Topics: {df_topics_display['Topic'].nunique()}")
print(f"\\n{df_topics_display.to_string(index=False)}")

df_topics_display.to_csv('topic_statistics.csv', index=False)
print(f"\\n✅ Topic statistics saved to: topic_statistics.csv")

unique_topics = df_topics_display['Topic'].tolist()


TOPIC ANALYSIS (ROS1 BAG)
\n🔍 Analyzing TF connections...
\n📊 Total Connections: 13
📊 Unique Topics: 10
\n                          Topic                         Message Type  Count                          Description  Frequency (Hz)
                            /tf               tf2_msgs/msg/TFMessage  20459   Static: base_footprint→laser_frame       44.172132
                            /tf               tf2_msgs/msg/TFMessage  17614 Static: base_footprint→base_imu_link       38.029617
                            /tf               tf2_msgs/msg/TFMessage  16294  SLAM: map→odom, odom→base_footprint       35.179663
                       /cmd_vel              geometry_msgs/msg/Twist  11418                                            24.652104
                           /imu                  sensor_msgs/msg/Imu   9553                                            20.625464
                            /tf               tf2_msgs/msg/TFMessage   8928        Odometry: odom→base_footprint       1

### 3. Message Frequency Distribution

In [ ]:
# Analyze message frequency distribution
print("=" * 60)
print("MESSAGE FREQUENCY DISTRIBUTION")
print("=" * 60)

with Reader(bag_path) as reader:
    connection_timestamps = defaultdict(list)  # Track by connection index
    topic_timestamps = defaultdict(list)  # Also track by topic for backward compatibility

    print("\\nCollecting timestamps...")
    for connection, timestamp, rawdata in tqdm(reader.messages(), total=reader.message_count):
        topic = connection.topic
        timestamp_sec = timestamp * 1e-9

        # Add to topic_timestamps
        topic_timestamps[topic].append(timestamp_sec)

        # Find connection index for detailed tracking
        conn_idx = None
        for idx, conn in enumerate(reader.connections):
            if (conn.topic == connection.topic and
                conn.msgtype == connection.msgtype and
                conn.msgcount == connection.msgcount):
                conn_idx = idx
                break

        if conn_idx is not None:
            connection_timestamps[conn_idx].append(timestamp_sec)

    print("\\n📈 Inter-Message Interval Statistics (by connection):")
    print(f"{'Topic':<40} {'Description':<30} {'Mean (ms)':<12} {'Std (ms)':<12}")
    print("-" * 94)

    for conn_idx in sorted(connection_timestamps.keys()):
        timestamps = connection_timestamps[conn_idx]
        if len(timestamps) > 1:
            intervals = np.diff(timestamps) * 1000

            # Get topic info from df_topics
            topic_row = df_topics[df_topics['Connection_Index'] == conn_idx].iloc[0]
            topic_name = topic_row['Topic']
            description = topic_row['Description'][:28] if topic_row['Description'] else ''

            print(f"{topic_name:<40} {description:<30} {np.mean(intervals):<12.2f} {np.std(intervals):<12.2f}")


MESSAGE FREQUENCY DISTRIBUTION
\nCollecting timestamps...


100%|██████████| 98205/98205 [00:01<00:00, 97074.03it/s] 

\n📈 Inter-Message Interval Statistics (by connection):
Topic                                    Description                    Mean (ms)    Std (ms)    
----------------------------------------------------------------------------------------------
/csi_cam_0/camera_info                                                  59.50        500.15      
/constraint_list                                                        2766.79      10945.50    
/csi_cam_0/image_raw/compressed                                         4480.40      3405.35     
/imu                                                                    46.95        447.40      
/odom                                                                   111.79       1323.80     
/scan                                                                   1427.87      3374.64     
/tf                                      SLAM: map→odom, odom→base_fo   27.52        258.56      
/tf                                      Static: base_footprint→la

### 4. Message Size Distribution Analysis

In [ ]:
# Analyze message sizes per connection
print("=" * 60)
print("MESSAGE SIZE ANALYSIS")
print("=" * 60)

with Reader(bag_path) as reader:
    connection_sizes = defaultdict(list)

    print("\\nCollecting message sizes...")
    for connection, timestamp, rawdata in tqdm(reader.messages(), total=reader.message_count):
        # Find connection index
        conn_idx = None
        for idx, conn in enumerate(reader.connections):
            if (conn.topic == connection.topic and
                conn.msgtype == connection.msgtype and
                conn.msgcount == connection.msgcount):
                conn_idx = idx
                break

        if conn_idx is not None:
            connection_sizes[conn_idx].append(len(rawdata))

    # Calculate statistics
    size_stats = []
    for conn_idx, sizes in connection_sizes.items():
        sizes_array = np.array(sizes)
        topic_row = df_topics[df_topics['Connection_Index'] == conn_idx].iloc[0]

        size_stats.append({
            'Topic': topic_row['Topic'],
            'Description': topic_row['Description'][:40] if topic_row['Description'] else '',
            'Avg Size (bytes)': np.mean(sizes_array),
            'Total Size (MB)': np.sum(sizes_array) / (1024 * 1024)
        })

    df_sizes = pd.DataFrame(size_stats)
    df_sizes = df_sizes.sort_values('Total Size (MB)', ascending=False)

    print("\\n📏 Message Size Statistics:")
    print(df_sizes.to_string(index=False))

    df_sizes.to_csv('message_size_statistics.csv', index=False)


MESSAGE SIZE ANALYSIS
\nCollecting message sizes...


100%|██████████| 98205/98205 [00:00<00:00, 102015.44it/s]


\n📏 Message Size Statistics:
                          Topic                          Description  Avg Size (bytes)  Total Size (MB)
/csi_cam_0/image_raw/compressed                                           35867.980000         3.420637
                           /imu                                             325.000000         2.960896
                            /tf  SLAM: map→odom, odom→base_footprint        181.000000         2.812590
                          /scan                                            9239.000000         2.775464
                          /odom                                             719.000000         2.751681
          /trajectory_node_list                                            6978.941463         2.728811
         /csi_cam_0/camera_info                                             361.000000         2.680192
                            /tf   Static: base_footprint→laser_frame        107.000000         2.087701
               /constraint_list    

In [ ]:
# Generate ROS Bag Basic EDA summary report
print("=" * 60)
print("ROS BAG BASIC EDA - SUMMARY REPORT")
print("=" * 60)

rosbag_summary = f"""
📊 ROS BAG BASIC EDA SUMMARY
{'='*60}

FILE INFORMATION:
  • File: {bag_name}
  • Size: {file_size_mb:.2f} MB
  • Duration: {duration_sec:.2f} seconds ({duration_sec/60:.2f} minutes)

MESSAGE STATISTICS:
  • Total Messages: {total_messages:,}
  • Unique Topics: {len(unique_topics)}
  • Message Types: {df_topics['Message Type'].nunique()}
  • Average Rate: {avg_message_rate:.2f} msg/sec

DATA QUALITY:
  • Average Data Rate: {file_size_mb / (duration_sec/60):.2f} MB/min

TOP 3 TOPICS BY SIZE:
"""

for idx, row in df_sizes.head(3).iterrows():
    rosbag_summary += f"  {idx+1}. {row['Topic']}: {row['Total Size (MB)']:.2f} MB\n"

print(rosbag_summary)

# Save report
with open('rosbag_basic_eda_summary.txt', 'w') as f:
    f.write(rosbag_summary)

print("\n✅ ROS Bag Basic EDA summary saved to: rosbag_basic_eda_summary.txt")

ROS BAG BASIC EDA - SUMMARY REPORT

📊 ROS BAG BASIC EDA SUMMARY

FILE INFORMATION:
  • File: session_data.bag
  • Size: 34.35 MB
  • Duration: 463.17 seconds (7.72 minutes)

MESSAGE STATISTICS:
  • Total Messages: 98,205
  • Unique Topics: 13
  • Message Types: 9
  • Average Rate: 212.03 msg/sec

DATA QUALITY:
  • Average Data Rate: 4.45 MB/min

TOP 3 TOPICS BY SIZE:
  3. /csi_cam_0/image_raw/compressed: 3.42 MB
  4. /imu: 2.96 MB
  7. /tf: 2.81 MB


✅ ROS Bag Basic EDA summary saved to: rosbag_basic_eda_summary.txt
